# Experiment 5: Scattering Features + CNN Model




In [76]:
import sys
sys.path.append('../src')
import warnings
warnings.filterwarnings("ignore") 

from utils.reduce import reduce_pca
from utils.split import train_test_split, train_test_split_pytorch
from utils.UltrasoundDataset import UltrasoundDataset
from utils.Networks import  BasicBlock, Scattering2dResNet
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
from torch.autograd import Variable
from kymatio.torch import Scattering2D
import argparse
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms
import torch.optim
import pickle
import pandas as pd
import numpy as np
import mlflow
import matplotlib.pyplot as plt

In [77]:
# create an experiment in mlruns if doesn't exit and specifiy where to log
mlflow.set_experiment('scattering_cnn_experiment')

INFO: 'scattering_cnn_experiment' does not exist. Creating a new experiment


## Upload Ultrasound images

In [78]:
with open('../data/02_interim/bmodes_steatosis_assessment_IJCARS.pickle', 'rb') as handle:
    df = pickle.load(handle)

In [79]:
M, N= 434, 636 # ultrasound image dimension

In [80]:
# split training and test (by making sure the 10 ultrasound images of one patient is in the same set)
train_data, test_data = train_test_split(df)
train_data, val_data = train_test_split(train_data)

In [81]:
###############################################################################
# If a GPU is available, let's use it!
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
###############################################################################

# Create dataset

In [82]:
# create dataset
# from https://github.com/python-engineer/pytorchTutorial/blob/master/09_dataloader.py

train_dataset = UltrasoundDataset(train_data)
test_dataset  = UltrasoundDataset(test_data)
val_dataset  = UltrasoundDataset(val_data)

In [83]:
def conv3x3(in_planes, out_planes, stride=1):
    "3x3 convolution with padding"
    # in_planes = in_channels
    # out_planes = out_channels
    return nn.Conv2d(in_planes, out_planes, kernel_size=10, stride=stride,
                     padding=1, bias=False)


## Defining scattering transformations

In [84]:
# Set the parameters by cross-validation
from itertools import product
param_batch_size = [10, 20, 50]
param_J = [2,3]
param_max_order = [1,2]
params = list(product(param_batch_size,param_J, param_max_order))


## Training and Testing Functions

In [85]:
def train(model, device, train_loader, optimizer, epoch, scattering):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(scattering(data))
        loss = F.cross_entropy(output, target.type(torch.long))
        loss.backward()
        optimizer.step()
        if batch_idx % 20 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))

def test(model, device, test_loader, scattering):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(scattering(data))
            test_loss += F.cross_entropy(output, target.type(torch.long), reduction='sum').item() # sum up batch loss
            pred = output.max(1, keepdim=True)[1] # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    print('\nValidation set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))
    return test_loss, 100. * correct / len(test_loader.dataset)


## Training

In [ ]:
for param in params:
    # Do cross-validation
    with mlflow.start_run():
        batch_size = param[0]
        J = param[1]
        max_order = param[2]
        mlflow.log_param('batch_size',batch_size)
        mlflow.log_param('J', J)
        mlflow.log_param('max_order', max_order)

        #create data loader
        train_loader = DataLoader(dataset=train_dataset,
                                  batch_size=batch_size,
                                  shuffle=True,
                                  num_workers=2)
        test_loader = DataLoader(dataset=test_dataset,
                                  batch_size=batch_size,
                                  shuffle=True,
                                  num_workers=2)
        val_loader = DataLoader(dataset=val_dataset,
                                  batch_size=batch_size,
                                  shuffle=True,
                                  num_workers=2)
        

        # compute scattering features---------------------------------------                       
        if max_order == 1:
            scattering = Scattering2D(J=J, shape=(M, N), max_order=max_order)
            K = 17*1
        else:
            scattering = Scattering2D(J=J, shape=(M, N))
            K = 81*1

        if use_cuda:
            scattering = scattering.cuda()     
        
        model = Scattering2dResNet(K,2).to(device)

        #training------------------------------------------------------------                         
        from tqdm import tqdm
        # Optimizer
        lr = 0.1
        val_loss_no_imp = 0
        val_loss_last = 0
        patience = 5
        for epoch in tqdm(range(0, 50)):
            optimizer = torch.optim.SGD(model.parameters(), lr=lr, momentum=0.9,
                                      weight_decay=0.0005)
            lr*=0.2
            train(model, device, train_loader, optimizer, epoch+1, scattering)
            val_loss , _ = test(model, device, val_loader, scattering)
            if val_loss > val_loss_last: 
                val_loss_no_imp +=1
            else:
                val_loss_no_imp = 0
            val_loss_last = val_loss
            if val_loss_no_imp >= patience: break
            
        val_loss , val_accuracy = test(model, device, val_loader, scattering)
        mlflow.log_metric('validation_accuracy',val_accuracy)  
        print('Combinaison Done')






  0%|          | 0/50 [00:00<?, ?it/s]

Train Epoch: 1 [0/440 (0%)]	Loss: 0.656500
Train Epoch: 1 [200/440 (45%)]	Loss: 0.688136
Train Epoch: 1 [400/440 (91%)]	Loss: 1.076102




  2%|▏         | 1/50 [00:11<08:59, 11.00s/it]


Validation set: Average loss: 0.7052, Accuracy: 30/50 (60.00%)

Train Epoch: 2 [0/440 (0%)]	Loss: 0.706304
Train Epoch: 2 [200/440 (45%)]	Loss: 0.572892
Train Epoch: 2 [400/440 (91%)]	Loss: 0.491799




  4%|▍         | 2/50 [00:22<08:50, 11.06s/it]

...


Validation set: Average loss: 0.3943, Accuracy: 50/50 (100.00%)

Train Epoch: 43 [0/440 (0%)]	Loss: 0.536912
Train Epoch: 43 [200/440 (45%)]	Loss: 0.471888
Train Epoch: 43 [400/440 (91%)]	Loss: 0.385349




 86%|████████▌ | 43/50 [10:59<01:47, 15.35s/it]


Validation set: Average loss: 0.3976, Accuracy: 49/50 (98.00%)

Train Epoch: 44 [0/440 (0%)]	Loss: 0.451599
Train Epoch: 44 [200/440 (45%)]	Loss: 0.433924
Train Epoch: 44 [400/440 (91%)]	Loss: 0.435235




 88%|████████▊ | 44/50 [11:14<01:32, 15.35s/it]


Validation set: Average loss: 0.3954, Accuracy: 50/50 (100.00%)

Train Epoch: 45 [0/440 (0%)]	Loss: 0.507237


# Run Mlflow to see results

`!mlflow ui`

Should launch something like this:



In [ ]:
# !mlflow ui

In [47]:
test_data['id'].unique()

array([ 4,  9, 28, 33, 52, 53], dtype=uint8)